# 1. Import Dependencies

In [205]:
!pip install stable-baselines3[extra]
import gym 
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


# 2. Load Environment

In [206]:
environment_name = "CartPole-v0"

In [207]:
env = gym.make(environment_name)

In [208]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    while not done:
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:49.0
Episode:2 Score:35.0
Episode:3 Score:11.0
Episode:4 Score:15.0
Episode:5 Score:11.0


# 3. Understanding The Environment


In [209]:
# 0-push cart to left, 1-push cart to the right
env.action_space.sample()


0

In [210]:
# [cart position, cart velocity, pole angle, pole angular velocity]
env.observation_space.sample()

array([-2.8915381e+00,  2.9394483e+38, -1.2395046e-01, -3.3691952e+38],
      dtype=float32)

# 3. Train an RL Model

In [211]:
log_path = os.path.join('Training','Logs')

In [212]:
log_path

'Training/Logs'

In [213]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose = 1)

Using cuda device


In [214]:
model.learn(total_timesteps=20000)

-----------------------------
| time/              |      |
|    fps             | 833  |
|    iterations      | 1    |
|    time_elapsed    | 2    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 589         |
|    iterations           | 2           |
|    time_elapsed         | 6           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.007813473 |
|    clip_fraction        | 0.101       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | -0.00449    |
|    learning_rate        | 0.0003      |
|    loss                 | 7.63        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0163     |
|    value_loss           | 55.4        |
-----------------------------------------
----------------------------------

# 4. Save and Reload Model

In [215]:
PPO_path = os.path.join('Training', 'Saved Models', 'PPO_model')

In [216]:
model.save(PPO_path)

In [163]:
del model

In [165]:
model = PPO.load('PPO_model', env=env)

FileNotFoundError: [Errno 2] No such file or directory: 'PPO_Path.zip'

# 5. Evaluation


In [217]:
evaluate_policy(model, env, n_eval_episodes=10,)

(200.0, 0.0)

In [218]:
env.close()

# 6. Test Model

In [219]:
obs = env.reset()
while True:
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    if done: 
        print('info', info)
        break

info [{'TimeLimit.truncated': True, 'terminal_observation': array([-0.15251863,  0.38164228, -0.02301168, -0.38050777], dtype=float32)}]


# 7. Viewing Logs in Tensorboard

In [220]:
training_log_path = os.path.join(log_path, 'PPO_3')

# 8. Adding a callback to the training Stage

In [221]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold
import os
save_path = os.path.join('Training', 'Saved Models')
log_path = os.path.join('Training', 'Logs')
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=190, verbose=1)
eval_callback = EvalCallback(env, 
                             callback_on_new_best=stop_callback, 
                             eval_freq=10000, 
                             best_model_save_path=save_path, 
                             verbose=1)
model = PPO('MlpPolicy', env, verbose = 1,)
model.learn(total_timesteps=20000, callback=eval_callback)
model_path = os.path.join('Training', 'Saved Models', 'best_model')
model = PPO.load(model_path, env=env)
evaluate_policy(model, env, n_eval_episodes=10,)
env.close()

Using cuda device
-----------------------------
| time/              |      |
|    fps             | 818  |
|    iterations      | 1    |
|    time_elapsed    | 2    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 572         |
|    iterations           | 2           |
|    time_elapsed         | 7           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.009031538 |
|    clip_fraction        | 0.102       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | 0.000646    |
|    learning_rate        | 0.0003      |
|    loss                 | 6.8         |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0165     |
|    value_loss           | 48.9        |
-----------------------------------------
----------------

# 9. Changing Policies


In [222]:
net_arch=[dict(pi=[128, 128, 128, 128], vf=[128, 128, 128, 128])]
model = PPO('MlpPolicy', env, verbose = 1, policy_kwargs={'net_arch': net_arch})
model.learn(total_timesteps=20000, callback=eval_callback)

Using cuda device
-----------------------------
| time/              |      |
|    fps             | 691  |
|    iterations      | 1    |
|    time_elapsed    | 2    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 454         |
|    iterations           | 2           |
|    time_elapsed         | 9           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.013793856 |
|    clip_fraction        | 0.194       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.682      |
|    explained_variance   | 0.00332     |
|    learning_rate        | 0.0003      |
|    loss                 | 2.55        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0228     |
|    value_loss           | 18.8        |
-----------------------------------------
----------------

# 10. Using an Alternate Algorithm

In [223]:
from stable_baselines3 import DQN
model = DQN('MlpPolicy', env, verbose = 1,)
model.learn(total_timesteps=20000, callback=eval_callback)
dqn_path = os.path.join('Training', 'Saved Models', 'DQN_model')
model.save(dqn_path)
model = DQN.load(dqn_path, env=env)
evaluate_policy(model, env, n_eval_episodes=10,)
env.close()

Using cuda device
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.954    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 9178     |
|    time_elapsed     | 0        |
|    total_timesteps  | 97       |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.916    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 8814     |
|    time_elapsed     | 0        |
|    total_timesteps  | 176      |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.854    |
| time/               |          |
|    episodes         | 12       |
|    fps              | 8971     |
|    time_elapsed     | 0        |
|    total_timesteps  | 308      |
----------------------------------
----------------------------------
| 

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 108      |
|    fps              | 8649     |
|    time_elapsed     | 0        |
|    total_timesteps  | 2420     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 112      |
|    fps              | 8628     |
|    time_elapsed     | 0        |
|    total_timesteps  | 2480     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 116      |
|    fps              | 8618     |
|    time_elapsed     | 0        |
|    total_timesteps  | 2549     |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 216      |
|    fps              | 8635     |
|    time_elapsed     | 0        |
|    total_timesteps  | 4838     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 220      |
|    fps              | 8628     |
|    time_elapsed     | 0        |
|    total_timesteps  | 4896     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 224      |
|    fps              | 8628     |
|    time_elapsed     | 0        |
|    total_timesteps  | 4976     |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 324      |
|    fps              | 8617     |
|    time_elapsed     | 0        |
|    total_timesteps  | 7095     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 328      |
|    fps              | 8613     |
|    time_elapsed     | 0        |
|    total_timesteps  | 7155     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 332      |
|    fps              | 8613     |
|    time_elapsed     | 0        |
|    total_timesteps  | 7254     |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 432      |
|    fps              | 8569     |
|    time_elapsed     | 1        |
|    total_timesteps  | 9205     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 436      |
|    fps              | 8567     |
|    time_elapsed     | 1        |
|    total_timesteps  | 9280     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 440      |
|    fps              | 8571     |
|    time_elapsed     | 1        |
|    total_timesteps  | 9375     |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 532      |
|    fps              | 8325     |
|    time_elapsed     | 1        |
|    total_timesteps  | 11458    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 536      |
|    fps              | 8323     |
|    time_elapsed     | 1        |
|    total_timesteps  | 11516    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 540      |
|    fps              | 8327     |
|    time_elapsed     | 1        |
|    total_timesteps  | 11613    |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 640      |
|    fps              | 8365     |
|    time_elapsed     | 1        |
|    total_timesteps  | 13764    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 644      |
|    fps              | 8364     |
|    time_elapsed     | 1        |
|    total_timesteps  | 13832    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 648      |
|    fps              | 8360     |
|    time_elapsed     | 1        |
|    total_timesteps  | 13885    |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 748      |
|    fps              | 8416     |
|    time_elapsed     | 1        |
|    total_timesteps  | 16272    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 752      |
|    fps              | 8416     |
|    time_elapsed     | 1        |
|    total_timesteps  | 16348    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 756      |
|    fps              | 8415     |
|    time_elapsed     | 1        |
|    total_timesteps  | 16410    |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 856      |
|    fps              | 8452     |
|    time_elapsed     | 2        |
|    total_timesteps  | 18739    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 860      |
|    fps              | 8454     |
|    time_elapsed     | 2        |
|    total_timesteps  | 18835    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 864      |
|    fps              | 8452     |
|    time_elapsed     | 2        |
|    total_timesteps  | 18899    |
----------------------------------
----------------------------------
| rollout/          